In [3]:
import os
import pickle
import sklearn
import pandas as pd
import numpy as np
# os.chdir('/Users/dethf/NYU/Spring 2019/Chunara Internship/nyu-twipsy/')

In [4]:
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))

/home/centos/anaconda3/envs/yijun/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.18.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/centos/anaconda3/envs/yijun/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.18.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/centos/anaconda3/envs/yijun/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/centos/anaconda3/envs/yijun/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 0.18.1 when usi

Processing data.

In [6]:
with open('/mnt/volume/result/juliana_allSF_2016_04.json', 'r') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object. I want to convert it into an *array* of JSON objects 
# which, in and of itself, is one large JSON object basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects separated by a comma.
data_json_str = "[" + ','.join(data) + "]"

df = pd.read_json(data_json_str)

In [53]:
# df.columns

In [54]:
# df.head(3)

In [55]:
# print(len(df))

In [56]:
# df.text[:10]

In [11]:
df_text = df[['text']] # instead of df.text, which will only return Series object.

In [57]:
# df_text.head(20)

Now feed the data into classifier to filter out those tobacco tweets.

In [13]:
# running the classifier
tob_preds = clf_tob.predict(df_text)

In [14]:
len(tob_preds)

395757

In [16]:
# finding out how many tweets there were in each group
# numpy.unique(tob_preds, return_counts=True)
print("NOT-Tobacco:", np.unique(tob_preds, return_counts=True)[1][0],
      "Tobacco:",np.unique(tob_preds, return_counts=True)[1][1])

NOT-Tobacco: 394627 Tobacco: 1130


In [17]:
indexTobacco = np.where(tob_preds == 'Tobacco')
indexTobacco = indexTobacco[0].tolist()
len(indexTobacco)
# indexTobacco

1130

In [18]:
# Double checked the index from tob_preds is correct.
# for i in indexTobacco:
#     print(tob_preds[i])

In [58]:
# tob_preds[:20]

In [20]:
dfTobacco = df.iloc[indexTobacco,]

In [59]:
# dfTobacco.head(3)

In [60]:
# for i in dfTobacco.text[:30]:
#     print(i)

-----------------------------------
Remove all the word 'cleaning' in the data and predict.

In [74]:
def preprocessAndPredictAndSave(year, month):
    print('%s - %s' %(year, month))
    
    # preprocess
    with open('/mnt/volume/result/juliana_allSF_'+year+'_'+month+'.json', 'r') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data_no_new_line = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object. I want to convert it into an *array* of JSON objects 
    # which, in and of itself, is one large JSON object basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects separated by a comma.
    data_json_str = "[" + ','.join(data_no_new_line) + "]"
    df = pd.read_json(data_json_str)
    
    df_text = df[['text']]
    df_text_filtered = df_text.replace(to_replace ="Cleaning", value ="", regex=True)
    
    # running the classifier
    tob_preds_filtered = clf_tob.predict(df_text_filtered)
    
    indexTobacco_filtered = np.where(tob_preds_filtered == 'Tobacco')
    indexTobacco_filtered = indexTobacco_filtered[0].tolist()
    
    dfTobacco_filtered = df_text_filtered.iloc[indexTobacco_filtered,]
    
    
    # Write to file
    SF_tobacco_FileName = '/mnt/volume/result/juliana_allSF_tobacco_'+year+'_'+month+'.json'
    with open(SF_tobacco_FileName, "w", encoding="utf-8") as f:
        for i in indexTobacco_filtered:
            f.write(data[i])
            
    number_total = np.unique(tob_preds_filtered, return_counts=True)
    number_not_tobacco = number_total[1][0]
    number_tobacco = number_total[1][1]
    number_tobacco_file = len(dfTobacco_filtered)
    if number_tobacco != number_tobacco_file:
        print('should be equal')
        return -1
    print("Total:", number_not_tobacco+number_tobacco,
          "Tobacco:",number_tobacco_file)
    return number_tobacco_file

In [75]:
# len(indexTobacco_filtered)
# with open('/mnt/volume/result/juliana_allSF_'+year+'_'+month+'.json', 'r') as f:
#     data = f.readlines()
# for i in indexTobacco_filtered:
#     print(data[i])
#     break

In [ ]:
number_tobacco_all = []
for year in ['2016']:
    for month in ['04','05','06','07','08','09','10','11','12']:
        number_tobacco_month = preprocessAndPredictAndSave(year, month)
        number_tobacco_all.append(number_tobacco_month)
for year in ['2017', '2018']:
    for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        number_tobacco_month = preprocessAndPredictAndSave(year, month)
        number_tobacco_all.append(number_tobacco_month)
for year in ['2019']:
    for month in ['01','02','03','04']:
        number_tobacco_month = preprocessAndPredictAndSave(year, month)
        number_tobacco_all.append(number_tobacco_month)
print(number_tobacco_all)

2016 - 04


In [43]:
df_text_filtered = df_text.replace(to_replace ="Cleaning", value ="", regex=True)
df_text_filtered.head(20)

,text
0,HAPPY FOUR TWON E MY AMERICANS \n&amp; THE WO...
1,@bobby trying to do any kind of animation in k...
2,I would say 1 in 4 people walking on the stree...
3,@sstvrkk ya right I'm ur only fan lms if u agree
4,@Clarko you don’t get nice animations for free...
5,"@IJEeditorial 38😭days😭since😭our😭publication, b..."
6,Shooting videos with @videografiko &amp; benny...
7,@DaleWright42 I hatewatch.
8,This #Nursing #job might be a great fit for yo...
9,@BlckdByJilZarin should be fun but my husband ...


Now we trying to input the data after filtered into classifier.

In [44]:
# running the classifier
tob_preds_filtered = clf_tob.predict(df_text_filtered)

In [45]:
# finding out how many tweets there were in each group
# numpy.unique(tob_preds, return_counts=True)
print("NOT-Tobacco:", np.unique(tob_preds_filtered, return_counts=True)[1][0],
      "Tobacco:",np.unique(tob_preds_filtered, return_counts=True)[1][1])

NOT-Tobacco: 394850 Tobacco: 907


In [46]:
indexTobacco_filtered = np.where(tob_preds_filtered == 'Tobacco')
indexTobacco_filtered = indexTobacco_filtered[0].tolist()
len(indexTobacco_filtered)
# indexTobacco

907

In [47]:
dfTobacco_filtered = df_text_filtered.iloc[indexTobacco_filtered,]
dfTobacco_filtered.head(3)

,text
172,enjoyed a nice cup of andytownsf coffee while ...
354,@halfbatch i'll smoke double for you AK 👌🖖
978,All the sudden everyone trynna skip to smoke t...


In [48]:
for i in dfTobacco_filtered.text:
    print(i)

enjoyed a nice cup of andytownsf coffee while sun bathing outside @thefamilyroomsf contemplating… https://t.co/2IdGi3KK6r
@halfbatch i'll smoke double for you AK 👌🖖
All the sudden everyone trynna skip to smoke today 😂 weak ass lil kids man
Smoking cookie clouds like they lavender
acknowledgment and disavowal
Cigarettes are so disgusting
i sneaked out last night to go vape with sam and nate
Im not smoking outside today either
@sfmta_muni Haight and Divis
Pathways Home Health and Hospice: Physical... (#SouthSanFranciscoUS, CA) https://t.co/Ao22iCyCMq #Healthcare #Job https://t.co/q31tXWeZz1
Also the smell of the ocean and salt water and just being home is really nice right now.
Why y'all skipping school to smoke? Smoke before &amp; after lol
Isn't this weekend the Calistoga Wine and Food festival @SolageCalistoga ??? Pre party !! #diningaround https://t.co/HXKxEnMNjJ
@partychronicles com certeza! E já começamos :) #tamojunto
Kylo Ren and Stimpy #MakeTVShowsEvil @midnight
Hospice Aide - P